In [1]:
import os
import pandas as pd
import ssl
import json

from kafka import KafkaProducer
import time

ssl._create_default_https_context = ssl._create_unverified_context

New topic

In [2]:
from kafka.admin import KafkaAdminClient, NewTopic
KAFKA_BOOTSTRAP_SERVERS='10.67.22.100:9092'
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    )

In [3]:
kafka_admin.list_topics()


['topic_stream',
 '__consumer_offsets',
 'topic_test',
 'topic_result',
 'topic_results']

In [4]:
#a_new_topic = NewTopic(name='topic_test', 
#                       num_partitions=2, 
#                       replication_factor=1)

#kafka_admin.create_topics(new_topics=[a_new_topic])

In [5]:
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [6]:
url = "https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/MAPD_miniDT_stream/data_000019.txt"
df = pd.read_csv(url)

In [27]:
%%time
jlist = []
for i in range(1000,2000):
    jj = df[df.HEAD !=2].iloc[i].to_dict()
    for key in jj:
        if key!="TDC_MEAS":
            jj[key] = int(jj[key])
    jlist.append(jj)

#time.sleep(2)
#for i in range(1000):
#    jj = df[df.HEAD !=2].iloc[i].to_dict()
#    for key in jj:
#        if key!="TDC_MEAS":
#            jj[key] = int(jj[key])
#    producer.send('topic_stream', json.dumps(jj).encode('utf-8'))
#producer.flush()

CPU times: user 4.03 s, sys: 0 ns, total: 4.03 s
Wall time: 4.02 s


In [28]:
%%time
for i in range(1000):
    producer.send('topic_stream', json.dumps(jlist[i]).encode('utf-8'))
producer.flush()

CPU times: user 90.1 ms, sys: 0 ns, total: 90.1 ms
Wall time: 87.9 ms


In [49]:
jj = df[df.HEAD !=2].iloc[100:110].to_dict()
for key in jj:
    if key!="TDC_MEAS":
        jj[key] = list(map(int,jj[key]))
producer.send('topic_stream', json.dumps(jj).encode('utf-8'))
producer.flush()

In [11]:
#for i in range(0,81):
#    i = str(i).zfill(2)
#    url = f"https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/MAPD_miniDT_stream/data_0000{i}.txt"
#    df = pd.read_csv(url)
#    for j in range(0,df.shape[0]//1000):
#        producer.send('a_partitioned_topic', json.dumps(msg).encode('utf-8'))
#        producer.flush()
#        time.sleep(0.25)
#        